In [1]:
model_name = 'Llama-2-7b-chat-dropout0.01_skip_anchor'

In [2]:
import json
with open(f'hex-phi/{model_name}.json', 'r', encoding='utf-8') as file:
    harmful_kl_list = json.load(file)
with open(f'sampled_330_alpaca_anchor/{model_name}.json', 'r', encoding='utf-8') as file:
    helpful_kl_list = json.load(file)

In [3]:
helpful_kl_list[0]

{'component_idx': 109,
 'component_type': 'mlp',
 'layer_idx': 1,
 'head_idx': 22,
 'kl': 234.54315185546875}

In [4]:
component_idx_to_harmful_kl_list = {}
for harmful_kl in harmful_kl_list:
    component_idx = harmful_kl['component_idx']
    component_idx_to_harmful_kl_list[component_idx] = harmful_kl
    
component_idx_to_helpful_kl_list = {}
for helpful_kl in helpful_kl_list:
    component_idx = helpful_kl['component_idx']
    component_idx_to_helpful_kl_list[component_idx] = helpful_kl

In [5]:
harmful_sub_helpful_kl_list = []
for component_idx, harmful_kl in component_idx_to_harmful_kl_list.items():
    component_type = harmful_kl['component_type']
    layer_idx = harmful_kl['layer_idx']
    head_idx = harmful_kl['head_idx']
    kl = harmful_kl['kl'] - component_idx_to_helpful_kl_list[component_idx]['kl']
    harmful_sub_helpful_kl_list.append({
        'component_idx': component_idx,
        'component_type': component_type,
        'layer_idx': layer_idx,
        'head_idx': head_idx,
        'kl': kl
    })
    
harmful_sub_helpful_kl_list = sorted(harmful_sub_helpful_kl_list, key=lambda x: x['kl'], reverse=True)

with open(f'combined/{model_name}.json', 'w') as json_file:
    json.dump(harmful_sub_helpful_kl_list, json_file, indent=4, ensure_ascii=False)
